<a href="https://colab.research.google.com/github/johanjan/MOOC-HPFEM-source/blob/master/MOOC_HPFEM_ErrorEstimate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### MOOC-HPFEM: Template notebook to be used to complete the assignments



The present notebook is meant to be a template for you to use while doing the assignments of the course. The first cell is meant to be run as-is and takes care of configuring the environment for you. The second cell is empty by default and is where you are supposed to paste the code you are given in the assignments so that you can modify and run it.

### FEniCS DFS formulation to run and modify yourself

In [0]:
#@title
# Setup system environment and define utility functions and variables
from google.colab import files
try:
    from dolfin import *; from mshr import *
except ImportError as e:
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !sed -e 's:artful:bionic:' /etc/apt/sources.list.d/fenics-packages-ubuntu-fenics-artful.list > temp
    !mv temp /etc/apt/sources.list.d/fenics-packages-ubuntu-fenics-artful.list
    !sed -e 's:artful:bionic:' /etc/apt/sources.list > temp
    !mv temp /etc/apt/sources.list
    !apt-get update -qq
    !apt install -y --no-install-recommends fenics    
    from dolfin import *; from mshr import *
    
import matplotlib.pyplot as plt
from IPython.display import clear_output, display, update_display
import time
import dolfin.common.plotting as fenicsplot 

def plot_compact(u, t, stepcounter, QQ, pl, ax): # Compact plot utility function
  if stepcounter == 0:
    pl, ax = plt.subplots(); display(pl); clear_output(); # Plotting setup
  if stepcounter % 5 == 0:
    #u.set_allow_extrapolation(True)
    uEuclidnorm = project(sqrt(inner(u, u)), QQ)
    ax.cla()
    fig = plt.gcf()
    fig.set_size_inches(16, 4)
    plt.subplot(1, 2, 1)
    pp = plot(uEuclidnorm, cmap="coolwarm")
    plt.title("Solution at t=%f" % (t)) # Plot norm of solution
    if t == 0:
      plt.axis(G)
      plt.colorbar(pp, shrink=0.5)
    plt.subplot(1, 2, 2);
    if t == 0:
      plot(QQ.mesh())
      plt.title("Mesh") # Plot mesh
    plt.tight_layout(); dpl = display(pl, display_id="test");
  
  return (pl, ax)

def plot_compact_static(u, QQ, label = "Velocity"): # Compact plot utility function
  pl, ax = plt.subplots();
  uEuclidnorm = project(sqrt(inner(u, u)), QQ); ax.cla(); fig = plt.gcf(); fig.set_size_inches(16, 4)
  plt.subplot(1, 2, 1); pp = plot(uEuclidnorm, cmap="coolwarm"); plt.title("%s" % (label)) # Plot norm of velocity
  plt.axis(G); plt.colorbar(pp, shrink=0.5); 
  plt.subplot(1, 2, 2);
  plot(QQ.mesh()); plt.title("Mesh") # Plot mesh
  plt.tight_layout(); dpl = display(pl, display_id="test");
  
  return (pl, ax)



In [0]:
esolution=10
mesh = UnitSquareMesh(resolution, resolution)

def AllBoundary(x, on_boundary):
    return on_boundary

zero = Constant(0.0)

def aform(u, v):
    return inner(grad(u), grad(v))

def Lform(f, v):
    return f*v

### Primal problem
f = Expression("32.*x[0]*(1. - x[0])+32.*x[1]*(1. - x[1])", domain=mesh, degree=5)
ue = Expression("16.*x[0]*(1. - x[0])*x[1]*(1. - x[1])", domain=mesh, degree=5)

Qp = FunctionSpace(mesh,'CG',1);
bcp = DirichletBC(Qp, zero, AllBoundary)

u = TrialFunction(Qp);
v = TestFunction(Qp);

ap = aform(u, v)*dx 
Lp = Lform(f, v)*dx 

U = Function(Qp)
solve(ap == Lp, U, bcp)

### Dual problem
Qd = FunctionSpace(mesh, 'CG', 2);
psi = Constant(1.0)
bcd = DirichletBC(Qd, zero, AllBoundary)

w = TestFunction(Qd);
phi = TrialFunction(Qd);
ad = aform(w, phi)*dx 
Ld = Lform(psi, w)*dx
phi = Function(Qd)
solve(ad == Ld, phi, bcd)
